# 09 - Validação Temporal (Time-Based Validation)

Este notebook implementa a validação temporal para simular o cenário real do desafio.

## Estratégia:
- **Conjunto de Treino**: Semanas 1 até ~47 de 2022
- **Conjunto de Validação**: Últimas 5 semanas de 2022 
- **Teste**: 5 semanas de Janeiro 2023 (submissão final)

Esta divisão simula o cenário real onde usamos dados históricos para prever o futuro.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import warnings
warnings.filterwarnings('ignore')

print('📅 Iniciando Validação Temporal para Simulação Real do Desafio')
print('🎯 Objetivo: Criar conjuntos de treino e validação que simulam o cenário real')

# Criar pasta submissao3 se não existir
os.makedirs('../data/submissao3', exist_ok=True)
print('📁 Pasta data/submissao3 criada/verificada')

📅 Iniciando Validação Temporal para Simulação Real do Desafio
🎯 Objetivo: Criar conjuntos de treino e validação que simulam o cenário real
📁 Pasta data/submissao3 criada/verificada


## 1. Carregamento dos Dados Brutos de 2022

In [2]:
# Carregar dados brutos de transações
print('📂 Carregando dados brutos de 2022...')

transacoes = pd.read_parquet(
    '../data/part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet'
)

print(f'📊 Dados carregados: {transacoes.shape}')
print(f'📊 Período: {transacoes["transaction_date"].min()} até {transacoes["transaction_date"].max()}')

# Renomear colunas para padronização
transacoes = transacoes.rename(columns={
    'internal_store_id': 'pdv_id',
    'internal_product_id': 'produto_id',
    'transaction_date': 'data',
    'quantity': 'quantidade',
    'gross_value': 'faturamento'
})

# Converter data para datetime
transacoes['data'] = pd.to_datetime(transacoes['data'])

print('✅ Dados de transação preparados')

📂 Carregando dados brutos de 2022...
📊 Dados carregados: (6560698, 11)
📊 Período: 2022-01-01 até 2022-12-31
✅ Dados de transação preparados


## 2. Agregação Semanal e Identificação de Semanas

In [3]:
# Criar coluna de semana
print('📅 Criando agregação semanal...')

transacoes['semana'] = transacoes['data'].dt.to_period('W-MON').dt.start_time

# Agregação semanal
agregacao_semanal = transacoes.groupby(['semana', 'pdv_id', 'produto_id']).agg({
    'quantidade': 'sum',
    'faturamento': 'sum',
    'distributor_id': 'first'
}).reset_index()

print(f'📊 Agregação semanal: {agregacao_semanal.shape}')

# Verificar distribuição de semanas
semanas_unicas = sorted(agregacao_semanal['semana'].unique())
print(f'📊 Semanas disponíveis: {len(semanas_unicas)}')
print(f'📊 Primeira semana: {semanas_unicas[0]}')
print(f'📊 Última semana: {semanas_unicas[-1]}')

# Mostrar as últimas 10 semanas para identificar as 5 finais
print('\n📋 Últimas 10 semanas de 2022:')
for i, semana in enumerate(semanas_unicas[-10:]):
    registros = agregacao_semanal[agregacao_semanal['semana'] == semana].shape[0]
    print(f'   {i+1:2d}. {semana.strftime("%Y-%m-%d")} - {registros:,} registros')

print('✅ Análise temporal concluída')

📅 Criando agregação semanal...
📊 Agregação semanal: (6241315, 6)
📊 Semanas disponíveis: 53
📊 Primeira semana: 2021-12-28 00:00:00
📊 Última semana: 2022-12-27 00:00:00

📋 Últimas 10 semanas de 2022:
    1. 2022-10-25 - 120,399 registros
    2. 2022-11-01 - 125,855 registros
    3. 2022-11-08 - 132,270 registros
    4. 2022-11-15 - 136,781 registros
    5. 2022-11-22 - 79,583 registros
    6. 2022-11-29 - 124,445 registros
    7. 2022-12-06 - 127,147 registros
    8. 2022-12-13 - 135,561 registros
    9. 2022-12-20 - 103,611 registros
   10. 2022-12-27 - 92,730 registros
✅ Análise temporal concluída


## 3. Divisão Temporal: Treino vs Validação

In [4]:
# Definir ponto de corte: últimas 5 semanas para validação
print('✂️ Definindo divisão temporal...')

# Últimas 5 semanas para validação
semanas_validacao = semanas_unicas[-5:]
semanas_treino = semanas_unicas[:-5]

print(f'🏋️ Conjunto de TREINO: {len(semanas_treino)} semanas')
print(f'   📅 Período: {semanas_treino[0].strftime("%Y-%m-%d")} até {semanas_treino[-1].strftime("%Y-%m-%d")}')

print(f'\n🔍 Conjunto de VALIDAÇÃO: {len(semanas_validacao)} semanas')
print(f'   📅 Período: {semanas_validacao[0].strftime("%Y-%m-%d")} até {semanas_validacao[-1].strftime("%Y-%m-%d")}')

print('\n📋 Semanas de validação:')
for i, semana in enumerate(semanas_validacao):
    print(f'   {i+1}. {semana.strftime("%Y-%m-%d")}')

# Filtrar dados por período
dados_treino = agregacao_semanal[agregacao_semanal['semana'].isin(semanas_treino)].copy()
dados_validacao = agregacao_semanal[agregacao_semanal['semana'].isin(semanas_validacao)].copy()

print(f'\n📊 Estatísticas da divisão:')
print(f'   🏋️ Dados de treino: {dados_treino.shape[0]:,} registros')
print(f'   🔍 Dados de validação: {dados_validacao.shape[0]:,} registros')
print(f'   📊 Proporção: {dados_treino.shape[0]/(dados_treino.shape[0]+dados_validacao.shape[0])*100:.1f}% treino / {dados_validacao.shape[0]/(dados_treino.shape[0]+dados_validacao.shape[0])*100:.1f}% validação')

print('✅ Divisão temporal definida')

✂️ Definindo divisão temporal...
🏋️ Conjunto de TREINO: 48 semanas
   📅 Período: 2021-12-28 até 2022-11-22

🔍 Conjunto de VALIDAÇÃO: 5 semanas
   📅 Período: 2022-11-29 até 2022-12-27

📋 Semanas de validação:
   1. 2022-11-29
   2. 2022-12-06
   3. 2022-12-13
   4. 2022-12-20
   5. 2022-12-27

📊 Estatísticas da divisão:
   🏋️ Dados de treino: 5,657,821 registros
   🔍 Dados de validação: 583,494 registros
   📊 Proporção: 90.7% treino / 9.3% validação
✅ Divisão temporal definida


## 4. Grid Completo: Garantir Todas as Combinações

In [5]:
# Criar grid completo para ambos os conjuntos
print('🎯 Criando grid completo para treino e validação...')

# Identificar todas as combinações únicas de PDV x Produto que aparecem nos dados
combinacoes_unicas = agregacao_semanal[['pdv_id', 'produto_id']].drop_duplicates()
print(f'📊 Combinações únicas PDV x Produto: {len(combinacoes_unicas):,}')

def criar_grid_completo_super_otimizado(semanas, combinacoes_unicas, dados_reais, nome_conjunto):
    """
    Criar grid completo para um conjunto de semanas de forma super otimizada para memória e performance.
    """
    print(f'\n🔄 Criando grid para {nome_conjunto} (VERSÃO SUPER OTIMIZADA)...')

    # DataFrame com as semanas desejadas
    semanas_df = pd.DataFrame({'semana': semanas})

    # Usar merge com 'how="cross"' para criar o produto cartesiano apenas entre
    # as semanas e as combinações PDV-Produto válidas.
    # Isso evita a explosão de memória e é extremamente rápido.
    grid_df = semanas_df.merge(combinacoes_unicas, how='cross')
    print(f'   📊 Grid criado: {len(grid_df):,} registros (tamanho correto)')

    # Merge com os dados de transações reais
    grid_completo = grid_df.merge(
        dados_reais[['semana', 'pdv_id', 'produto_id', 'quantidade', 'faturamento', 'distributor_id']],
        on=['semana', 'pdv_id', 'produto_id'],
        how='left'
    )

    # Preencher valores ausentes com 0
    grid_completo['quantidade'] = grid_completo['quantidade'].fillna(0).astype('int32')
    grid_completo['faturamento'] = grid_completo['faturamento'].fillna(0).astype('float32')

    # Preencher distributor_id usando forward e backward fill (agrupado por produto)
    # Isso garante que todas as combinações de um produto tenham um distribuidor associado
    grid_completo['distributor_id'] = grid_completo.groupby('produto_id')['distributor_id'].transform('ffill').transform('bfill').astype('float32')
    
    # Validação final do preenchimento de distributor_id (opcional, mas bom para garantir)
    if grid_completo['distributor_id'].isnull().any():
        print("   ⚠️ Alerta: Ainda existem distributor_id nulos. Preenchendo com um valor padrão (-1).")
        grid_completo['distributor_id'] = grid_completo['distributor_id'].fillna(-1)

    # Estatísticas
    zeros = (grid_completo['quantidade'] == 0).sum()
    nao_zeros = (grid_completo['quantidade'] > 0).sum()

    print(f'   📊 {nome_conjunto} - Shape final: {grid_completo.shape}')
    print(f'   📊 Zeros: {zeros:,} ({zeros/len(grid_completo)*100:.1f}%)')
    print(f'   📊 Não-zeros: {nao_zeros:,} ({nao_zeros/len(grid_completo)*100:.1f}%)')

    return grid_completo

# Criar grids completos com a versão super otimizada
train_data = criar_grid_completo_super_otimizado(semanas_treino, combinacoes_unicas, dados_treino, "TREINO")
validation_data = criar_grid_completo_super_otimizado(semanas_validacao, combinacoes_unicas, dados_validacao, "VALIDAÇÃO")

print('✅ Grids completos criados')

🎯 Criando grid completo para treino e validação...
📊 Combinações únicas PDV x Produto: 1,044,310

🔄 Criando grid para TREINO (VERSÃO SUPER OTIMIZADA)...
   📊 Grid criado: 50,126,880 registros (tamanho correto)
   📊 TREINO - Shape final: (50126880, 6)
   📊 Zeros: 44,549,560 (88.9%)
   📊 Não-zeros: 5,543,608 (11.1%)

🔄 Criando grid para VALIDAÇÃO (VERSÃO SUPER OTIMIZADA)...
   📊 Grid criado: 5,221,550 registros (tamanho correto)
   📊 VALIDAÇÃO - Shape final: (5221550, 6)
   📊 Zeros: 4,645,877 (89.0%)
   📊 Não-zeros: 571,729 (10.9%)
✅ Grids completos criados


## 5. Validação da Divisão Temporal

In [6]:
# Validar que não há sobreposição temporal
print('🔍 Validando divisão temporal...')

# Verificar datas
max_data_treino = train_data['semana'].max()
min_data_validacao = validation_data['semana'].min()

print(f'📅 Última semana de treino: {max_data_treino}')
print(f'📅 Primeira semana de validação: {min_data_validacao}')
print(f'📅 Gap temporal: {(min_data_validacao - max_data_treino).days} dias')

# Verificar consistência
assert max_data_treino < min_data_validacao, "❌ ERRO: Sobreposição temporal detectada!"
print('✅ Validação temporal: SEM sobreposição')

# Verificar combinações
combos_treino = set(zip(train_data['pdv_id'], train_data['produto_id']))
combos_validacao = set(zip(validation_data['pdv_id'], validation_data['produto_id']))

intersecao = combos_treino.intersection(combos_validacao)
print(f'🔍 Combinações em comum: {len(intersecao):,} ({len(intersecao)/len(combos_treino)*100:.1f}%)')

# Estatísticas finais
print('\n📊 RESUMO DA VALIDAÇÃO TEMPORAL:')
print('=' * 50)
print(f'🏋️ TREINO:')
print(f'   • Semanas: {len(semanas_treino)}')
print(f'   • Registros: {len(train_data):,}')
print(f'   • Período: {train_data["semana"].min().strftime("%Y-%m-%d")} até {train_data["semana"].max().strftime("%Y-%m-%d")}')

print(f'\n🔍 VALIDAÇÃO:')
print(f'   • Semanas: {len(semanas_validacao)}')
print(f'   • Registros: {len(validation_data):,}')
print(f'   • Período: {validation_data["semana"].min().strftime("%Y-%m-%d")} até {validation_data["semana"].max().strftime("%Y-%m-%d")}')

print('\n🎯 Esta divisão simula perfeitamente o cenário do desafio:')
print('   • Usamos dados históricos (treino) para prever o futuro (validação)')
print('   • Mantemos a sequência temporal')
print('   • Testamos no mesmo formato da submissão final')

print('✅ Validação temporal concluída com sucesso!')

🔍 Validando divisão temporal...
📅 Última semana de treino: 2022-11-22 00:00:00
📅 Primeira semana de validação: 2022-11-29 00:00:00
📅 Gap temporal: 7 dias
✅ Validação temporal: SEM sobreposição
🔍 Combinações em comum: 1,044,310 (100.0%)

📊 RESUMO DA VALIDAÇÃO TEMPORAL:
🏋️ TREINO:
   • Semanas: 48
   • Registros: 50,126,880
   • Período: 2021-12-28 até 2022-11-22

🔍 VALIDAÇÃO:
   • Semanas: 5
   • Registros: 5,221,550
   • Período: 2022-11-29 até 2022-12-27

🎯 Esta divisão simula perfeitamente o cenário do desafio:
   • Usamos dados históricos (treino) para prever o futuro (validação)
   • Mantemos a sequência temporal
   • Testamos no mesmo formato da submissão final
✅ Validação temporal concluída com sucesso!


## 6. Salvamento dos Conjuntos

In [7]:
# Salvar conjuntos de treino e validação
print('💾 Salvando conjuntos de treino e validação...')

# Salvar em data/submissao3
train_data.to_parquet('../data/submissao3/train_data.parquet', index=False)
validation_data.to_parquet('../data/submissao3/validation_data.parquet', index=False)

print('✅ Arquivos salvos:')
print('   • data/submissao3/train_data.parquet')
print('   • data/submissao3/validation_data.parquet')

# Salvar metadados da divisão
import pickle

metadados_divisao = {
    'data_criacao': pd.Timestamp.now(),
    'estrategia': 'Time-Based Validation',
    'semanas_treino': len(semanas_treino),
    'semanas_validacao': len(semanas_validacao),
    'periodo_treino': f"{train_data['semana'].min()} até {train_data['semana'].max()}",
    'periodo_validacao': f"{validation_data['semana'].min()} até {validation_data['semana'].max()}",
    'registros_treino': len(train_data),
    'registros_validacao': len(validation_data),
    'combinacoes_unicas': len(combinacoes_unicas),
    'objetivo': 'Simular cenário real do desafio - dados históricos prevendo futuro'
}

with open('../data/submissao3/time_based_validation_metadata.pkl', 'wb') as f:
    pickle.dump(metadados_divisao, f)

print('📋 Metadados salvos em: data/submissao3/time_based_validation_metadata.pkl')

print('\n🎉 VALIDAÇÃO TEMPORAL CONCLUÍDA!')
print('=' * 60)
print('🎯 A partir de agora, use SEMPRE estes arquivos:')
print('   • train_data.parquet para treinar modelos')
print('   • validation_data.parquet para avaliar performance')
print('\n📈 Esta será sua "bússola" para saber se as mudanças funcionam!')
print('💡 Qualquer melhoria deve ser validada neste conjunto antes da submissão final.')

💾 Salvando conjuntos de treino e validação...
✅ Arquivos salvos:
   • data/submissao3/train_data.parquet
   • data/submissao3/validation_data.parquet
📋 Metadados salvos em: data/submissao3/time_based_validation_metadata.pkl

🎉 VALIDAÇÃO TEMPORAL CONCLUÍDA!
🎯 A partir de agora, use SEMPRE estes arquivos:
   • train_data.parquet para treinar modelos
   • validation_data.parquet para avaliar performance

📈 Esta será sua "bússola" para saber se as mudanças funcionam!
💡 Qualquer melhoria deve ser validada neste conjunto antes da submissão final.
